# Running Your Code on a Quantum Computer

This notebook demonstrates how to run a quantum circuit on an IBM Quantum computer using Qiskit Runtime.


## Setting Up Qiskit Runtime

First, we authenticate using an IBM Quantum token to access real quantum hardware. 
Make sure to replace `<my_ibm_quantum_token>` with your actual token or use `getpass.getpass()` to avoid exposing it in the notebook.


In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
 

my_token="<my_ibm_quantum_token>"
service = QiskitRuntimeService(channel="ibm_quantum", token=my_token)

## Listing Available Backends

We fetch the list of available quantum computers (backends) to see which ones are online and operational.


In [ ]:
service.backends()

## Creating and Running a Quantum Circuit

We construct a simple quantum circuit with two qubits:
- Apply a **Hadamard (H) gate** on qubit 0.
- Apply a **CNOT (CX) gate** to entangle qubit 0 with qubit 1.
- Add **measurements** to both qubits.

This circuit creates a **Bell state**, which is a fundamental entangled state in quantum computing.


In [ ]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import  SamplerV2 as Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# Create empty circuit
example_circuit = QuantumCircuit(2)
example_circuit.h(0)
example_circuit.cx(0, 1)

example_circuit.measure_all()
example_circuit.draw('mpl')


## Selecting a Quantum Backend

We choose the least busy available quantum computer to run our circuit efficiently.


In [ ]:
# You'll need to specify the credentials when initializing QiskitRuntimeService, if they were not previously saved.
backend = service.least_busy(simulator=False, operational=True)
# backend = service.backend('ibm_kyiv')

## Optimizing the Circuit for Execution

Before sending the circuit to the quantum computer, we optimize it using Qiskit's preset pass manager, ensuring it runs efficiently on the selected backend.


In [ ]:
# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=0)
isa_circuit = pm.run(example_circuit)
 
isa_circuit.draw("mpl", idle_wires=False)

## Running the Circuit on IBM Quantum

We execute the circuit using Qiskit Runtime's `Sampler` and specify 3000 shots (repetitions) to collect measurement statistics.


In [ ]:
sampler = Sampler(backend)
job = sampler.run([isa_circuit], shots=3000)
print(f"job id: {job.job_id()}")


## Checking Job Status and Retrieving Results

Once the job is submitted, we check its status (e.g., `QUEUED`, `RUNNING`, `DONE`) and retrieve the final results.


In [ ]:
job.status()

In [ ]:
result = job.result()
print(result)